## Homework 4

This assignment focuses on methods of classification when data has a class imbalance. You will compare results obtained with Synthetic Minority Oversampling, ADASYN with FastKDE, and Normalizing Flows to oversample the minority class. To develop the data application, you will use



https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction.



This dataset is very large, and in this project, you may subset 10% of observations from each class. Your results should include confusion matrices and stratified K-fold validated estimates of accuracy and recalls.

In [3]:
import pandas as pd

#### This code should not be run again. It was used to subset the original data.

In [4]:
# importing train data
train = pd.read_csv('data/train.csv')
train

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1,10,0,0,0,0,0,...,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,...,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,...,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,...,4,1,4,2,0,1,1,1,0,0


In [5]:
# importing test data
test = pd.read_csv('data/test.csv')
test

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892811,1488022,0,1,6,0,0,0,1,0,0,...,4,2,3,4,0,1,0,0,1,0
892812,1488023,5,3,5,1,0,0,0,1,0,...,6,2,2,11,0,0,1,1,0,0
892813,1488024,0,1,5,0,0,1,0,0,0,...,5,2,2,11,0,1,1,0,0,0
892814,1488025,6,1,5,1,0,0,0,0,1,...,1,1,2,7,1,1,0,0,0,0


In [6]:
#subsetting the train data to only be 10% of each class and saving it over the original data
subset_train = train.groupby('target').sample(frac=0.1)
subset_train['target'].value_counts()
subset_train.to_csv('data/train.csv', index=False)
subset_train

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
211336,527987,0,2,1,8,0,0,0,0,0,...,3,1,1,7,1,1,0,1,0,0
18955,47519,0,3,1,3,0,0,1,0,0,...,3,2,3,5,0,0,1,1,1,1
375516,938513,0,1,1,1,0,0,1,0,0,...,1,1,1,4,0,1,1,0,0,1
111764,279774,0,0,1,2,0,0,1,0,0,...,6,0,5,3,0,1,1,0,0,0
92938,232653,0,3,1,2,1,0,0,1,0,...,4,0,5,11,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483055,1207113,1,1,1,7,0,4,0,0,1,...,3,1,1,9,0,0,1,0,1,0
485292,1212728,1,0,1,5,0,0,0,1,0,...,6,2,1,2,0,1,1,0,0,0
303247,757821,1,3,1,7,0,1,1,0,0,...,3,1,3,8,0,0,0,1,0,0
350300,875118,1,0,3,5,1,0,0,1,0,...,7,2,4,6,0,1,1,0,0,0


In [7]:
# subset to only be a random 10% and saving it over the original data because of github size limits
subset_test = test.sample(frac=0.1, random_state=440)
subset_test.to_csv('data/test.csv', index=False)
subset_test

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
761573,1269765,5,4,7,1,0,0,0,1,0,...,5,1,3,7,1,0,0,1,1,0
87010,144752,2,1,5,1,0,0,0,1,0,...,6,2,3,5,0,0,1,0,0,0
62560,104118,5,1,5,1,6,0,1,0,0,...,3,0,3,7,0,0,0,0,0,0
689487,1149369,2,1,5,0,0,0,0,0,1,...,3,1,5,7,0,1,1,0,0,0
413575,689431,2,2,3,0,0,1,0,0,0,...,4,1,4,14,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885394,1475801,3,1,11,0,0,0,1,0,0,...,7,0,2,12,0,1,1,0,1,0
318207,530568,0,1,3,0,0,1,0,0,0,...,6,2,3,9,0,0,0,0,0,0
790303,1317536,5,1,2,0,0,1,0,0,0,...,6,0,5,6,1,0,0,1,0,0
329478,549319,0,1,6,0,0,0,1,0,0,...,7,2,2,9,0,1,0,0,1,0
